<center>
<h1>Data with Danny - Data Bank Solutions</h1>
</center>
</center>

#### I have approached this case study using DuckDB by running SQL queries directly within Jupyter Notebooks. This method allows for seamless analysis without the conventional step of transferring tables to a dedicated database for querying. 

## Installing DuckDB and Pandas

In [1]:
!pip install duckdb
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 47.5 MB/s eta 0:00:00:00:0100:01


## Importing Libraries and Datasets

In [2]:
import duckdb
import pandas as pd

In [3]:
cn = pd.read_csv('/kaggle/input/data-bank-sample/customer_nodes.csv')
ct = pd.read_csv('/kaggle/input/data-bank-sample/customer_transcations.csv')
re = pd.read_csv('/kaggle/input/data-bank-sample/regions.csv')

## Data Cleaning and Preprocessing 

#### Upon analyzing the dataset, I realized that the primary task at hand is to convert the 'end_date', 'start_date', and 'txt_date' columns to datetime columns. This conversion is crucial to enable date-based calculations and comparisons, which are essential for extracting meaningful insights from the data. By converting these columns to datetime format, we can perform date-related operations, such as calculating date differences, identifying date patterns, and aggregating data by date ranges.

In [4]:
cn['start_date'] = pd.to_datetime(cn['start_date'], format='%m-%d-%y')
cn['end_date'] = pd.to_datetime(cn['end_date'], format='%m-%d-%y')
ct['txt_date'] = pd.to_datetime(ct['txt_date'], format='%m-%d-%y')

## Creating DuckDB Tables From Pandas Dataframes

In [5]:
duckdb.sql("CREATE TABLE customer_nodes AS SELECT * FROM cn")

In [6]:
duckdb.sql("CREATE TABLE customer_transactions AS SELECT * FROM ct")

In [7]:
duckdb.sql("CREATE TABLE regions AS SELECT * FROM re")

## Taking A Look At Contents Of The Tables

In [8]:
duckdb.sql("SELECT * FROM customer_nodes LIMIT 10")

┌─────────────┬───────────┬─────────┬─────────────────────┬─────────────────────┐
│ customer_id │ region_id │ node_id │     start_date      │      end_date       │
│    int64    │   int64   │  int64  │    timestamp_ns     │    timestamp_ns     │
├─────────────┼───────────┼─────────┼─────────────────────┼─────────────────────┤
│           1 │         3 │       4 │ 2020-01-02 00:00:00 │ 2020-01-03 00:00:00 │
│           2 │         3 │       5 │ 2020-01-03 00:00:00 │ 2020-01-17 00:00:00 │
│           3 │         5 │       4 │ 2020-01-27 00:00:00 │ 2020-02-18 00:00:00 │
│           4 │         5 │       4 │ 2020-01-07 00:00:00 │ 2020-01-19 00:00:00 │
│           5 │         3 │       3 │ 2020-01-15 00:00:00 │ 2020-01-23 00:00:00 │
│           6 │         1 │       1 │ 2020-01-11 00:00:00 │ 2020-02-06 00:00:00 │
│           7 │         2 │       5 │ 2020-01-20 00:00:00 │ 2020-02-04 00:00:00 │
│           8 │         1 │       2 │ 2020-01-15 00:00:00 │ 2020-01-28 00:00:00 │
│           9 │ 

In [44]:
duckdb.sql("SELECT * FROM customer_transactions LIMIT 10")

┌─────────────┬─────────────────────┬──────────┬────────────┐
│ customer_id │      txt_date       │ txn_type │ txn_amount │
│    int64    │    timestamp_ns     │ varchar  │   int64    │
├─────────────┼─────────────────────┼──────────┼────────────┤
│         429 │ 2020-01-21 00:00:00 │ deposit  │         82 │
│         155 │ 2020-01-10 00:00:00 │ deposit  │        712 │
│         398 │ 2020-01-01 00:00:00 │ deposit  │        196 │
│         255 │ 2020-01-14 00:00:00 │ deposit  │        563 │
│         185 │ 2020-01-29 00:00:00 │ deposit  │        626 │
│         309 │ 2020-01-13 00:00:00 │ deposit  │        995 │
│         312 │ 2020-01-20 00:00:00 │ deposit  │        485 │
│         376 │ 2020-01-03 00:00:00 │ deposit  │        706 │
│         188 │ 2020-01-13 00:00:00 │ deposit  │        601 │
│         138 │ 2020-01-11 00:00:00 │ deposit  │        520 │
├─────────────┴─────────────────────┴──────────┴────────────┤
│ 10 rows                                         4 columns │
└───────

In [45]:
duckdb.sql("SELECT * FROM regions LIMIT 10")

┌───────────┬─────────────┐
│ region_id │ region_name │
│   int64   │   varchar   │
├───────────┼─────────────┤
│         1 │ Australia   │
│         2 │ America     │
│         3 │ Africa      │
│         4 │ Asia        │
│         5 │ Europe      │
└───────────┴─────────────┘

<center>
<h1>A. Customer Nodes Exploration</h1>
</center>


## Q1. How many unique nodes are there on the Data Bank system?


In [9]:
# Unqiue Number of Nodes in Data Bank System

duckdb.sql("SELECT COUNT(DISTINCT node_id) as distinct_nodes FROM customer_nodes")

┌────────────────┐
│ distinct_nodes │
│     int64      │
├────────────────┤
│              5 │
└────────────────┘

## Q2. What is the number of nodes per region?


In [10]:
# No of nodes per region

duckdb.sql("SELECT re.region_name,COUNT(DISTINCT cn.node_id) as no_of_nodes FROM customer_nodes AS cn INNER JOIN regions as re ON cn.region_id = re.region_id GROUP BY re.region_name")

┌─────────────┬─────────────┐
│ region_name │ no_of_nodes │
│   varchar   │    int64    │
├─────────────┼─────────────┤
│ Africa      │           5 │
│ Europe      │           5 │
│ America     │           5 │
│ Australia   │           5 │
│ Asia        │           5 │
└─────────────┴─────────────┘

## Q3. How many customers are allocated to each region?


In [11]:
# Customers allocated to each region

duckdb.sql("SELECT re.region_name, COUNT(DISTINCT cn.customer_id) as customer_counts FROM customer_nodes AS cn INNER JOIN regions AS re ON cn.region_id = re.region_id GROUP BY re.region_name ORDER BY customer_counts DESC")

┌─────────────┬─────────────────┐
│ region_name │ customer_counts │
│   varchar   │      int64      │
├─────────────┼─────────────────┤
│ Australia   │             110 │
│ America     │             105 │
│ Africa      │             102 │
│ Asia        │              95 │
│ Europe      │              88 │
└─────────────┴─────────────────┘

## Q4. How many days on average are customers reallocated to a different node?


In [12]:
# Average number of days after which reallocation take's place

duckdb.sql("""

WITH customer_dates AS (
  SELECT
    customer_id,
    region_id,
    node_id,
    MIN(start_date) AS first_date
  FROM customer_nodes
  GROUP BY customer_id, region_id, node_id
),
reallocation_table AS (
  SELECT
    customer_id,
    node_id,
    region_id,
    first_date,
    DATEDIFF('day', first_date,
             LEAD(first_date) OVER(PARTITION BY customer_id
                                   ORDER BY first_date)) AS moving_days
  FROM customer_dates
)

SELECT
  ROUND(AVG(CAST(moving_days AS FLOAT)),0) AS avg_moving_days
FROM reallocation_table;

""")

┌─────────────────┐
│ avg_moving_days │
│     double      │
├─────────────────┤
│            24.0 │
└─────────────────┘

## Q5. What is the median, 80th and 95th percentile for this same reallocation days metric for each region?


In [13]:
duckdb.sql(
"""
WITH customer_dates AS (
  SELECT
    customer_id,
    region_id,
    node_id,
    MIN(start_date) AS first_date
  FROM customer_nodes
  GROUP BY customer_id, region_id, node_id
),

reallocation_table AS (
  SELECT
    node_id,
    region_id,
    first_date,
    DATEDIFF('day', first_date,
             LEAD(first_date) OVER(PARTITION BY customer_id
                                   ORDER BY first_date)) AS moving_days
  FROM customer_dates
),

percentiles AS(
  SELECT
    region_id,
    moving_days
  FROM reallocation_table
)

SELECT
  p.region_id,
  r.region_name,
  ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY p.moving_days),2) AS median, -- Added a comma after r.region_name
  ROUND(PERCENTILE_CONT(0.85) WITHIN GROUP (ORDER BY p.moving_days),2) AS p85,
  ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY p.moving_days),2) AS p90
FROM percentiles p
INNER JOIN regions r ON p.region_id = r.region_id
GROUP BY p.region_id,region_name


""")

┌───────────┬─────────────┬────────┬────────┬────────┐
│ region_id │ region_name │ median │  p85   │  p90   │
│   int64   │   varchar   │ double │ double │ double │
├───────────┼─────────────┼────────┼────────┼────────┤
│         2 │ America     │   21.0 │  37.65 │   45.2 │
│         5 │ Europe      │   22.0 │   37.9 │   44.0 │
│         4 │ Asia        │   22.0 │  37.55 │   44.0 │
│         1 │ Australia   │   22.0 │   37.2 │   46.8 │
│         3 │ Africa      │   21.0 │   36.2 │   45.8 │
└───────────┴─────────────┴────────┴────────┴────────┘

<center>
<h1>B. Customer Transactions</h1>
</center>

## What is the unique count and total amount for each transaction type?

In [14]:
duckdb.sql("""

SELECT txn_type, COUNT(txn_type) as unique_counts , SUM(txn_amount) as total_transaction_amount
FROM customer_transactions
GROUP BY  txn_type

""")

┌────────────┬───────────────┬──────────────────────────┐
│  txn_type  │ unique_counts │ total_transaction_amount │
│  varchar   │     int64     │          int128          │
├────────────┼───────────────┼──────────────────────────┤
│ purchase   │          1617 │                   806537 │
│ deposit    │          2671 │                  1359168 │
│ withdrawal │          1580 │                   793003 │
└────────────┴───────────────┴──────────────────────────┘

## What is the average total historical deposit counts and amounts for all customers?

In [15]:
duckdb.sql(
"""
WITH CTE AS (
SELECT 
customer_id,
AVG(txn_amount) as avg_deposit,
COUNT(*) as transaction_count
FROM customer_transactions
WHERE txn_type = 'deposit'
GROUP BY customer_id
)

SELECT 
ROUND(AVG(avg_deposit),2) as avg_deposit_amount,
ROUND(AVG(transaction_count),0) as avg_transactions
FROM CTE;


""")

┌────────────────────┬──────────────────┐
│ avg_deposit_amount │ avg_transactions │
│       double       │      double      │
├────────────────────┼──────────────────┤
│             508.61 │              5.0 │
└────────────────────┴──────────────────┘

## For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

In [16]:
duckdb.sql(
"""
WITH CTE1 AS (

SELECT date_part('month',TRY_CAST(txt_date AS DATE)) as month ,customer_id,txn_type,
COUNT(txn_type) FILTER(WHERE txn_type = 'deposit') OVER(PARTITION BY date_part('month',TRY_CAST(txt_date AS DATE)),customer_id) as deposit_count,
COUNT(txn_type) FILTER(WHERE txn_type = 'withdrawal') OVER(PARTITION BY date_part('month',TRY_CAST(txt_date AS DATE)),customer_id) as withdrawal_count,
COUNT(txn_type) FILTER(WHERE txn_type = 'purchase') OVER(PARTITION BY date_part('month',TRY_CAST(txt_date AS DATE)),customer_id) as purchase_count
FROM customer_transactions

),

CTE2 AS (

SELECT month, customer_id,
CASE
   WHEN deposit_count > 1 AND (withdrawal_count = 1 OR purchase_count = 1) THEN 'yes'
   ELSE 'no'
END AS verdict
FROM CTE1


)

SELECT month,COUNT(DISTINCT customer_id) as customer_count
FROM CTE2
WHERE verdict = 'yes'
GROUP BY month
ORDER BY month

""")


┌───────┬────────────────┐
│ month │ customer_count │
│ int64 │     int64      │
├───────┼────────────────┤
│     1 │            115 │
│     2 │            108 │
│     3 │            113 │
│     4 │             50 │
└───────┴────────────────┘

## What is the closing balance for each customer at the end of the month?

In [17]:
# closing balance for each customer at the end of the month
duckdb.sql(
""" 
WITH CTE AS (

SELECT  date_part('month', TRY_CAST(txt_date AS DATE)) as txn_month,
    date_part('year', TRY_CAST(txt_date AS DATE)) as txn_year,
    txt_date,
    customer_id,
    SUM((CASE WHEN txn_type ='deposit' THEN txn_amount ELSE 0 END) - (CASE WHEN txn_type <>'deposit' THEN txn_amount ELSE 0 END)) as balance
FROM customer_transactions   
GROUP BY date_part('month', TRY_CAST(txt_date AS DATE)) , date_part('year', TRY_CAST(txt_date AS DATE)),
txt_date, customer_id

),

Balances AS (

SELECT *,
     SUM(balance) OVER(PARTITION BY customer_id ORDER BY txt_date) as running_sum,
     ROW_NUMBER() OVER (PARTITION BY customer_id, txn_month, txn_year ORDER BY txt_date DESC) as rn
FROM CTE
ORDER BY txt_date

)

SELECT 
  customer_id,
  date_part('year', TRY_CAST(txt_date AS DATE)) * 100 + date_part('month', TRY_CAST(txt_date AS DATE)) as end_of_month,
  running_sum as closing_balance
FROM Balances
WHERE rn = 1
ORDER BY customer_id, txn_month; 


""")


┌─────────────┬──────────────┬─────────────────┐
│ customer_id │ end_of_month │ closing_balance │
│    int64    │    int64     │     int128      │
├─────────────┼──────────────┼─────────────────┤
│           1 │       202001 │             312 │
│           1 │       202003 │            -640 │
│           2 │       202001 │             549 │
│           2 │       202003 │             610 │
│           3 │       202001 │             144 │
│           3 │       202002 │            -821 │
│           3 │       202003 │           -1222 │
│           3 │       202004 │            -729 │
│           4 │       202001 │             848 │
│           4 │       202003 │             655 │
│           · │          ·   │              ·  │
│           · │          ·   │              ·  │
│           · │          ·   │              ·  │
│         498 │       202001 │            1360 │
│         498 │       202002 │            2195 │
│         498 │       202003 │            2989 │
│         498 │     

## What is the percentage of customers who increase their closing balance by more than 5%?

In [43]:
# closing balance increase by more than 5 %
duckdb.sql(
""" 
WITH CTE AS (

SELECT  date_part('month', TRY_CAST(txt_date AS DATE)) as txn_month,
    date_part('year', TRY_CAST(txt_date AS DATE)) as txn_year,
    txt_date,
    customer_id,
    SUM((CASE WHEN txn_type ='deposit' THEN txn_amount ELSE 0 END) - (CASE WHEN txn_type <>'deposit' THEN txn_amount ELSE 0 END)) as balance
FROM customer_transactions   
GROUP BY date_part('month', TRY_CAST(txt_date AS DATE)) , date_part('year', TRY_CAST(txt_date AS DATE)),
txt_date, customer_id

),

balances AS (

SELECT *,
     SUM(balance) OVER(PARTITION BY customer_id ORDER BY txt_date) as running_sum,
     ROW_NUMBER() OVER (PARTITION BY customer_id, txn_month, txn_year ORDER BY txt_date DESC) as rn
FROM CTE
ORDER BY txt_date

)

,closing_balances AS (

SELECT 
customer_id,
DATE_TRUNC('month', txt_date) + INTERVAL '1 month' - INTERVAL '1 day' AS end_of_month,
DATE_TRUNC('month', txt_date) - INTERVAL '1 day' AS previous_end_of_month,
running_sum as closing_balance
FROM balances
WHERE rn = 1
ORDER BY end_of_month

)

,percent_increase AS (

SELECT 
CB1.customer_id,
CB1.end_of_month,
CB1.closing_balance,
CB2.closing_balance as next_month_closing_balance,
(CB2.closing_balance / CB1.closing_balance) -1 as percentage_increase,
CASE WHEN (CB2.closing_balance > CB1.closing_balance AND 
(CB2.closing_balance / CB1.closing_balance) -1 > 0.05) THEN 1 ELSE 0 END as percentage_increase_flag
FROM CLOSING_BALANCES as CB1
INNER JOIN CLOSING_BALANCES as CB2 on CB1.end_of_month = CB2.previous_end_of_month 
AND CB1.customer_id = CB2.customer_id
WHERE CB1.closing_balance <> 0


)

SELECT 
SUM(percentage_increase_flag) / COUNT(percentage_increase_flag) as percentage_of_customers_increasing_balance
FROM percent_increase;


""")

┌────────────────────────────────────────────┐
│ percentage_of_customers_increasing_balance │
│                   double                   │
├────────────────────────────────────────────┤
│                        0.20966350301984468 │
└────────────────────────────────────────────┘

# THANK YOU !!!